# Screammers - Question A1:

It is well known that WRITING IN ALL CAPS ONLINE IS A SUBSTITUTE FOR SCREAMING… OR YELLING. *cough!*. (Or some might say it’s simply cruise control for cooooool). Write a job to find users that scream a lot, and provide a screamer score (a highly-technical metric that you will invent).

Import data from sampled_reddit dataset (10% of whole dataset)

In [13]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import year, month, dayofmonth, desc, length
import pyspark.sql.functions as func

df = sqlContext.read.json("hdfs://orion11:11001/sampled_reddit/*")
df.createOrReplaceTempView("Comments")
# Displays the content of the DataFrame to stdout
# df = originDF.select(originDF.author, originDF.body, length(originDF.body).alias("length"))

# originDF.printSchema()
df.printSchema()
print(df.count())
# df.groupBy(df.author).show()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- replies: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- saved: boolean (nullable = true)
 |--

Filter the comments, only select comments which contain only uppercase or special character and not wrote by bot/auto moderator

In [14]:
lowerCaseExpr = "([a-z])+"
botExpr = "[bB][oO][tT]"
filteredDF = (df
 .filter(~(df.body.rlike(lowerCaseExpr)) & ~(df.author.like("[deleted]") 
                                 | df.author.like("AutoModerator") 
                                 | df.author.rlike(botExpr)))
 .select(df.subreddit, df.author, df.body)

 )
filteredDF.count()

4452213

After fiterd the comments, group by author and count number of comments by author and total length of all the comments. So we can sort using total number of comments or sort by total comments length

In [15]:
authorComments = (filteredDF
 .groupBy(filteredDF.author)
 .agg(
     func.count(func.lit(1)).alias("Num Of scream"),
     func.sum(length(filteredDF.body)).alias("Screamer score")
     ))
authorComments.show()

+--------------------+-------------+--------------+
|              author|Num Of scream|Screamer score|
+--------------------+-------------+--------------+
|   EngineeringIsHard|           11|           311|
|          bobmystery|            8|           146|
|        trenchdigger|            1|            11|
| TasteTheRainbowHead|            3|            24|
| YesSomebodyElseDoes|           36|           108|
|         douglasr007|           19|           163|
|       SpikeKintarin|           16|           299|
|         galenspring|            3|            58|
|          VeiledAiel|            2|            19|
|        iHyperboloid|            2|            23|
|          Rab_Legend|           38|          1120|
|         CKlandSHARK|          125|          6455|
|        crimsonjella|           15|           368|
|         Spongyrocks|           56|          3009|
|4ever_alonelyfangirl|            5|            38|
|         tidder_bear|            2|           155|
|     drunkh

In [17]:
(authorComments
 .sort(desc("Num Of scream"))
 .show())

+--------------------+-------------+--------------+
|              author|Num Of scream|Screamer score|
+--------------------+-------------+--------------+
|          davidjl123|        16304|        145377|
|      atomicimploder|        15803|        127709|
|       TheNitromeFan|        11021|        121261|
|         Smartstocks|         9646|         80593|
|        Removedpixel|         8228|         70911|
|            Mooraell|         6838|         54638|
|RandomRedditorWithNo|         6342|         68055|
|        KingCaspianX|         5984|         64275|
|   SolidGoldMagikarp|         5911|         53155|
|            rideride|         5003|         38078|
|           cupofmilo|         4962|         39862|
|          xHOCKEYx12|         4642|         34065|
|             mfstyrf|         4407|        101392|
|              mokeru|         3986|        104452|
|           rschaosid|         3959|         28932|
|        FartyMcNarty|         3910|         35676|
|           

In [18]:
(authorComments
 .sort(desc("Screamer score"))
 .show())

+--------------------+-------------+--------------+
|              author|Num Of scream|Screamer score|
+--------------------+-------------+--------------+
|              ambral|          772|       2163636|
|        supergameman|          133|       1030986|
|               veeti|         2680|        908443|
|      roadrageroland|           77|        743513|
|     INFINITYISWEIRD|           60|        434716|
|          bhrgunatha|          259|        411979|
|Midnight_Rider_Daddy|           39|        349733|
|       Therianthrope|          204|        331477|
|      -TheTruthHurts|           57|        293383|
|         bleeaublaha|           29|        290000|
|       expiredcheese|           77|        265599|
|         TrenchFouch|          950|        259325|
|            Dragovic|           45|        231027|
|          __thiscall|           21|        210000|
|         DinoYouTube|           22|        187009|
|              Aragil|          875|        179978|
|          A

### For future reference (when we really want to get something off our chest), what are the top 5 subreddits for scream-y comments?
Instead of group by author, in this problem we will group all the comments by subreddit

In [19]:
subredditComments = (filteredDF
 .groupBy(filteredDF.subreddit)
 .agg(
     func.count(func.lit(1)).alias("Num Of scream")
     ))
subredditComments.show()

+--------------+-------------+
|     subreddit|Num Of scream|
+--------------+-------------+
|         anime|        10862|
|    MensRights|          791|
|  SaltLakeCity|          137|
|      lacrosse|           63|
|        HoJang|            2|
|        travel|          262|
|    QuotesPorn|          154|
|       Amateur|          201|
|    WahoosTipi|         2242|
|        Daleks|           27|
|          udel|            5|
|       ukraina|        47611|
|   battlefront|           36|
|           SCW|           12|
|   Jobopenings|            3|
|FreeHugsGaming|            2|
|   creepypasta|           54|
|  marvelheroes|          180|
|  electroswing|            4|
|        zipzip|            2|
+--------------+-------------+
only showing top 20 rows



Then we will sort the subreddit by number of scream comments

In [20]:
(subredditComments
 .sort(desc("Num of scream"))
 .show(5))

+---------+-------------+
|subreddit|Num Of scream|
+---------+-------------+
|AskReddit|       290342|
| counting|       231749|
| newsokur|       144728|
|      nfl|       106300|
|    funny|       103038|
+---------+-------------+
only showing top 5 rows

